# Phenotypes:
1. Hearing aids (f.3393)
2. Hearing difficulty/problems (f.2247)
3. Hearing difficulty/background noise (f.2257)

## Aim

Create a dataset of filtered individuals using the inclusion and exclusion criteria for diverse hearing related phenotyes to perform association analyses using the LMM.ipynb. 

## Location of files

In the shared folder is the original UKBB data
```
/SAY/dbgapstg/scratch/UKBiobank/phenotype_files/pleiotropy_R01/ukb42495_updatedJune2020
```

In my personal folder the filtered dataset

```
/home/dc2325/project/HI_UKBB
```

## Important phenotypic files

1. `200804_UKBB_HI_genotypeqc.csv` File containing all individuals that passed QC and hearing impairment variables
2. `200804_UKBB_HI_genotypeqc_excr.csv` File with applied exclusion criteria as indicated [here](https://docs.google.com/document/d/1cpxTzElpsEkwmBDjnMBHg2wW7CL1AcG_b0_0wE_k5rQ/edit). **Note**: this file excludes individuals with otosclerosis, Meniere's and other diseases, if you need to filter those particular phenotypes use file 1 instead.
3. `200811_UKBB_Tinnitus_plan1_2_3_f4803` File with filtered phenotypes for tinnitus plan 1,2 and 3 and imputed noise variables
4. `200814_UKBB_HI_genotypeqc_excr_impvars` Database with qc'ed individuals, exclusion criteria, noise imputed vars and tinnitus phenotypes

## Analysis plan

The phenotypes to be analyzed are the following:

1. Hearing aid user (f.3393)
"Do you use a hearing aid most of the time?"

2. Hearing difficulty/problems (f.2247)
"Do you have any difficulty with your hearing?"

3. Hearing difficulty/background noise (f.2257)
"Do you find it difficult to follow a conversation if there is background noise (such as TV, radio, children playing)?"

**Sex corresponds to f.22001 (genetic sex):**

- Male = 0
- Female = 1

**Noisy workplace and loud music exposure frequency: same as for Tinnitus**
                
1. Remove inconsistent individuals 
    - said 1,2 or 3 and in following visits said 0
    - said a higher exposure (e.g 3) and then a lower one (e.g 1 or 2) in following visits
2. Retain consistent individuals and use highest reported exposure

**The needs to be inverse normalized**

**Covariates to be included in the analysis include:**

1. Age at time of test (calculated from f.21003.0.0,f.21003.1.0,f.21003.2.0,f.21003.3.0)
2. Sex f.22001
3. Volume left ear f.4270 and right ear f.4277 (The volume set by the participant for the measurement which you are using in the analysis ir our case the last time they took the test). For the analysis we use the average of the right and left ear since there is overlap in the volume distribution
4. Noisy workplace f.4825
5. Loud music exposure f.4836


## Load libraries and set working dir

In [ ]:
#Load libraries
library(plyr)
library(tidyverse)
library(pander)
library(ggpubr)
library(rapportools)
library(ggplot2)
#Get working directory
getwd()

In [2]:
#Set working directory
setwd('~/project/HI_UKBB')

In [3]:
# Clean workspace
rm(list=ls())

In [4]:
df.final.imp = read.csv('200814_UKBB_HI_genotypeqc_excr_impvars.csv')

## Data summary and recode

In [5]:
dim(df.final.imp)

[1] 354347   1082

In [6]:
table(df.final.imp$f.22001)


Female   Male 
192414 161933 

In [7]:
df.final.imp$sex <- revalue(df.final.imp$f.22001.0.0, c("Male" = '0', 'Female'='1' ))
table(df.final.imp$sex)
dim(df.final.imp)


     1      0 
192414 161933 

[1] 354347   1083

# f.3393 Hearing aid: filter out inconsistent cases

In [8]:
# Recode function:
recode<-function(df,column_name){
  new_names<-c()
  for (i in column_name){
    new_column_name<-paste0(i,"_recode")
    new_names<-c(new_names,new_column_name)
    df[,new_column_name] <- revalue(df[,i], c("No"= 0, 
                                            "Yes" =1,
                                            "Prefer not to answer"= NA))
  }
  return (list(df=df,new_column_names=new_names))
}

# columns needs to be recoded:
column_name<-c("f.3393.0.0","f.3393.1.0","f.3393.2.0","f.3393.3.0")

# get a new data.frame with recoded columns added:
df.final.imp<-recode(df=df.final.imp,column_name)$df

# get names of recoded columns:
new_column_names<-recode(df=df.final.imp,column_name)$new_column_names

# show recode summary:
for (i in new_column_names)
{cat(i,"summary:");print(table(df.final.imp[,i]));cat("\n")}

f.3393.0.0_recode summary:
     0      1 
202311   9962 

f.3393.1.0_recode summary:
    0     1 
14472   878 

f.3393.2.0_recode summary:
    0     1 
32852  2853 

f.3393.3.0_recode summary:
   0    1 
2225  184 



In [9]:
dim(df.final.imp)

[1] 354347   1087

## f. 3393 Get patterns for possible answers

In [10]:
# Extract subset of data only with the recode columns of tinnitus
data_sub <- df.final.imp %>%
  select("IID",all_of(new_column_names)) 

# Function to extract all the available answers for 4 visits
# and put them in one string as "0000", "111", "991", etc

f2<-function(x){
  visit<-''
  for (i in 2:5){ # do not take the first column (i.e. IID)
    if (!is.na(x[i]))
    {visit<-paste0(visit,x[i])}
  }
  if(is.null(visit)){visit=NA}
  else{visit=visit}
  return (visit)
}

# Apply the above function and remove NAs
data_sub$visit<-apply(data_sub, 1, f2)

names(table(data_sub$visit))

[1] ""     "0"    "00"   "000"  "0000" "0001" "001"  "0010" "0011" "01"  
[11] "010"  "0100" "011"  "0110" "0111" "1"    "10"   "100"  "101"  "11"  
[21] "110"  "1100" "111"  "1111"

In [11]:
table(data_sub$visit)
all_pattern<-unique(data_sub$visit)


            0     00    000   0000   0001    001   0010   0011     01    010 
123790 191176  22628   3930    215     13    247      6     10   1830     24 
  0100    011   0110   0111      1     10    100    101     11    110   1100 
     1    192      1      4   9269     77     13     17    766      6      1 
   111   1111 
   125      6 

## f.3393 Get all the consistent cases

In [12]:
all_no<-all_pattern[which(grepl("0",all_pattern) & !grepl("1",all_pattern))]
all_yes<-all_pattern[which(!grepl("0",all_pattern) & grepl("1",all_pattern))]
table(data_sub[data_sub$visit%in%union(all_no,all_yes),"visit"])


     0     00    000   0000      1     11    111   1111 
191176  22628   3930    215   9269    766    125      6 

In [13]:
consistent_1<-c(all_no,all_yes)
yes_no<-all_pattern[which(grepl("0",all_pattern) & grepl("1",all_pattern))]
table(data_sub[data_sub$visit%in%yes_no,"visit"])


0001  001 0010 0011   01  010 0100  011 0110 0111   10  100  101  110 1100 
  13  247    6   10 1830   24    1  192    1    4   77   13   17    6    1 

In [14]:
might_inconsistent<-c(yes_no)

## f.3393 Remove inconsistent cases

In [15]:
# get all the inconsistent cases:
exceptions<-c("0001","001","0011","01","011","0111")
inconsistent<-setdiff(might_inconsistent,exceptions)

cat("There are",length(which(is.empty(data_sub$visit))),"NAs, removed")

There are 123790 NAs, removed

In [16]:
cat("There are",length(which(data_sub$visit%in%inconsistent)),"inconsistent cases, removed")

There are 146 inconsistent cases, removed

In [17]:
# removed those NAs and inconsistent cases: 
IID_hearing_aid<-data_sub %>%
  filter(!is.empty(visit)) %>%
  filter((!visit%in%inconsistent)) %>%
  select(IID)

dim(IID_hearing_aid)

[1] 230411      1

In [18]:
IID_hearing_aid<-IID_hearing_aid[,1]
cat("After removing all the NAs and inconsistent cases, there are",length(IID_hearing_aid)," individuals left")

After removing all the NAs and inconsistent cases, there are 230411  individuals left

In [19]:
# get the dataset after cleansing: 
df.hearing.aid<-df.final.imp %>%
  filter(IID%in%IID_hearing_aid)

dim(df.hearing.aid)

[1] 230411   1087

## f.3393 Step 1: classify cases and controls

In [20]:
hearing_aid <- df.hearing.aid %>% 
  select(IID,FID,f.22001.0.0, f.21003.0.0,f.21003.1.0,f.21003.2.0,f.21003.3.0,f.3393.0.0_recode,f.3393.1.0_recode,f.3393.2.0_recode,f.3393.3.0_recode)
head(hearing_aid)

,IID,FID,f.22001.0.0,f.21003.0.0,f.21003.1.0,f.21003.2.0,f.21003.3.0,f.3393.0.0_recode,f.3393.1.0_recode,f.3393.2.0_recode,f.3393.3.0_recode
,<int>,<int>,<fct>,<int>,<int>,<int>,<int>,<fct>,<fct>,<fct>,<fct>
1,1000019,1000019,Female,47,NA,NA,NA,0,NA,NA,NA
2,1000022,1000022,Male,53,NA,NA,NA,0,NA,NA,NA
3,1000035,1000035,Male,63,NA,NA,NA,0,NA,NA,NA
4,1000046,1000046,Female,62,NA,73,NA,NA,NA,0,NA
5,1000054,1000054,Female,65,NA,NA,NA,0,NA,NA,NA
6,1000063,1000063,Male,43,NA,NA,NA,0,NA,NA,NA


## Hearing aid f.3393

In [21]:
data_sub <- select(hearing_aid,starts_with("f.3393"))

# Function to define cases
f<-function(x){
  visit<-c()
  for (i in 1:4){
    if (!is.na(x[i]))
    {visit<-c(visit,x[i])}
  }
  if ("1" %in% visit){result= TRUE}
  else{result=FALSE}
  return (result)
}

# Apply the above function
hearing_aid$cases<-apply(data_sub, 1, f)
head(hearing_aid,10)

,IID,FID,f.22001.0.0,f.21003.0.0,f.21003.1.0,f.21003.2.0,f.21003.3.0,f.3393.0.0_recode,f.3393.1.0_recode,f.3393.2.0_recode,f.3393.3.0_recode,cases
,<int>,<int>,<fct>,<int>,<int>,<int>,<int>,<fct>,<fct>,<fct>,<fct>,<lgl>
1,1000019,1000019,Female,47,NA,NA,NA,0,NA,NA,NA,FALSE
2,1000022,1000022,Male,53,NA,NA,NA,0,NA,NA,NA,FALSE
3,1000035,1000035,Male,63,NA,NA,NA,0,NA,NA,NA,FALSE
4,1000046,1000046,Female,62,NA,73,NA,NA,NA,0,NA,FALSE
5,1000054,1000054,Female,65,NA,NA,NA,0,NA,NA,NA,FALSE
6,1000063,1000063,Male,43,NA,NA,NA,0,NA,NA,NA,FALSE
7,1000078,1000078,Female,52,57,60,NA,NA,0,0,NA,FALSE
8,1000105,1000105,Female,54,NA,NA,NA,0,NA,NA,NA,FALSE
9,1000112,1000112,Male,58,NA,68,NA,NA,NA,1,NA,TRUE


In [22]:
hearing_aid_cases <- hearing_aid %>% 
  filter(cases == TRUE)
head(hearing_aid_cases)
cat(dim(hearing_aid_cases)[1], "individuals are cases for f.3393")

,IID,FID,f.22001.0.0,f.21003.0.0,f.21003.1.0,f.21003.2.0,f.21003.3.0,f.3393.0.0_recode,f.3393.1.0_recode,f.3393.2.0_recode,f.3393.3.0_recode,cases
,<int>,<int>,<fct>,<int>,<int>,<int>,<int>,<fct>,<fct>,<fct>,<fct>,<lgl>
1,1000112,1000112,Male,58,NA,68,NA,NA,NA,1,NA,TRUE
2,1001067,1001067,Male,50,NA,NA,NA,1,NA,NA,NA,TRUE
3,1001384,1001384,Female,61,NA,NA,NA,1,NA,NA,NA,TRUE
4,1001459,1001459,Male,64,NA,NA,NA,1,NA,NA,NA,TRUE
5,1002548,1002548,Male,62,NA,NA,NA,1,NA,NA,NA,TRUE
6,1002888,1002888,Male,68,NA,NA,NA,1,NA,NA,NA,TRUE


12462 individuals are cases for f.3393

In [23]:
hearing_aid_controls <- hearing_aid %>% 
  filter(cases == FALSE)
head(hearing_aid_controls)
cat(dim(hearing_aid_controls)[1], "individuals are controls for f.3393")

,IID,FID,f.22001.0.0,f.21003.0.0,f.21003.1.0,f.21003.2.0,f.21003.3.0,f.3393.0.0_recode,f.3393.1.0_recode,f.3393.2.0_recode,f.3393.3.0_recode,cases
,<int>,<int>,<fct>,<int>,<int>,<int>,<int>,<fct>,<fct>,<fct>,<fct>,<lgl>
1,1000019,1000019,Female,47,NA,NA,NA,0,NA,NA,NA,FALSE
2,1000022,1000022,Male,53,NA,NA,NA,0,NA,NA,NA,FALSE
3,1000035,1000035,Male,63,NA,NA,NA,0,NA,NA,NA,FALSE
4,1000046,1000046,Female,62,NA,73,NA,NA,NA,0,NA,FALSE
5,1000054,1000054,Female,65,NA,NA,NA,0,NA,NA,NA,FALSE
6,1000063,1000063,Male,43,NA,NA,NA,0,NA,NA,NA,FALSE


217949 individuals are controls for f.3393

In [24]:
dim(hearing_aid)[1]-217949 -12462

[1] 0

## f.3393 Recode cases=1 and controls=0

In [25]:
hearing_aid$hearing_aid_cat <- as.integer(as.logical(hearing_aid$cases))
head(hearing_aid, 10)
# Get the number of NAs
#length(which(is.na(hearing_all$hearing_aid_cat)))

,IID,FID,f.22001.0.0,f.21003.0.0,f.21003.1.0,f.21003.2.0,f.21003.3.0,f.3393.0.0_recode,f.3393.1.0_recode,f.3393.2.0_recode,f.3393.3.0_recode,cases,hearing_aid_cat
,<int>,<int>,<fct>,<int>,<int>,<int>,<int>,<fct>,<fct>,<fct>,<fct>,<lgl>,<int>
1,1000019,1000019,Female,47,NA,NA,NA,0,NA,NA,NA,FALSE,0
2,1000022,1000022,Male,53,NA,NA,NA,0,NA,NA,NA,FALSE,0
3,1000035,1000035,Male,63,NA,NA,NA,0,NA,NA,NA,FALSE,0
4,1000046,1000046,Female,62,NA,73,NA,NA,NA,0,NA,FALSE,0
5,1000054,1000054,Female,65,NA,NA,NA,0,NA,NA,NA,FALSE,0
6,1000063,1000063,Male,43,NA,NA,NA,0,NA,NA,NA,FALSE,0
7,1000078,1000078,Female,52,57,60,NA,NA,0,0,NA,FALSE,0
8,1000105,1000105,Female,54,NA,NA,NA,0,NA,NA,NA,FALSE,0
9,1000112,1000112,Male,58,NA,68,NA,NA,NA,1,NA,TRUE,1


In [26]:
dim(hearing_aid)

[1] 230411     13

## Step 2: get the ages for hearing aids (f.3393)

In [27]:
# Get the subset data of cases
aid_age_cases = hearing_aid %>% 
  filter(hearing_aid_cat=="1")  %>%
  select(IID, f.3393.0.0_recode, f.3393.1.0_recode, f.3393.2.0_recode, f.3393.3.0_recode, f.21003.0.0, f.21003.1.0, f.21003.2.0, f.21003.3.0)
head(aid_age_cases,12)

,IID,f.3393.0.0_recode,f.3393.1.0_recode,f.3393.2.0_recode,f.3393.3.0_recode,f.21003.0.0,f.21003.1.0,f.21003.2.0,f.21003.3.0
,<int>,<fct>,<fct>,<fct>,<fct>,<int>,<int>,<int>,<int>
1,1000112,NA,NA,1,NA,58,NA,68,NA
2,1001067,1,NA,NA,NA,50,NA,NA,NA
3,1001384,1,NA,NA,NA,61,NA,NA,NA
4,1001459,1,NA,NA,NA,64,NA,NA,NA
5,1002548,1,NA,NA,NA,62,NA,NA,NA
6,1002888,1,NA,NA,NA,68,NA,NA,NA
7,1002944,1,NA,NA,NA,65,NA,NA,NA
8,1003258,0,1,NA,NA,69,74,NA,NA
9,1004012,1,NA,NA,NA,57,NA,NA,NA


In [28]:
# Get the # of column where first replied Yes:
aid_age_cases$visit_idx = apply(aid_age_cases, 1, function(x) unlist(which(x == '1')))
head(aid_age_cases)

,IID,f.3393.0.0_recode,f.3393.1.0_recode,f.3393.2.0_recode,f.3393.3.0_recode,f.21003.0.0,f.21003.1.0,f.21003.2.0,f.21003.3.0,visit_idx
,<int>,<fct>,<fct>,<fct>,<fct>,<int>,<int>,<int>,<int>,<list>
1,1000112,NA,NA,1,NA,58,NA,68,NA,4
2,1001067,1,NA,NA,NA,50,NA,NA,NA,2
3,1001384,1,NA,NA,NA,61,NA,NA,NA,2
4,1001459,1,NA,NA,NA,64,NA,NA,NA,2
5,1002548,1,NA,NA,NA,62,NA,NA,NA,2
6,1002888,1,NA,NA,NA,68,NA,NA,NA,2


In [29]:
# Define offset:
# offset: refers to the # of columns between the first age column (i.e.f.21003.0.0) and the first recode column (i.e.f.3393.0.0_recode)
offset = which(colnames(aid_age_cases) == 'f.21003.0.0') - which(colnames(aid_age_cases) == 'f.3393.0.0_recode')

# Define the function to extract the first time they said yes for cases 
f=get_age_func <- function(x) {
  visit_index=x[which(colnames(aid_age_cases)=="visit_idx")]
  index=min(unlist(visit_index))+offset
  age=x[index]
  final_age=unlist(age)
  if(is.null(final_age))
  {final_age<-NA}
  return(final_age)
}

# Get the final age for cases
aid_age_cases$age_final_aid = apply(aid_age_cases, 1, f)

# Show first 6 rows
head(aid_age_cases)
summary(aid_age_cases$age_final_aid)

,IID,f.3393.0.0_recode,f.3393.1.0_recode,f.3393.2.0_recode,f.3393.3.0_recode,f.21003.0.0,f.21003.1.0,f.21003.2.0,f.21003.3.0,visit_idx,age_final_aid
,<int>,<fct>,<fct>,<fct>,<fct>,<int>,<int>,<int>,<int>,<list>,<int>
1,1000112,NA,NA,1,NA,58,NA,68,NA,4,68
2,1001067,1,NA,NA,NA,50,NA,NA,NA,2,50
3,1001384,1,NA,NA,NA,61,NA,NA,NA,2,61
4,1001459,1,NA,NA,NA,64,NA,NA,NA,2,64
5,1002548,1,NA,NA,NA,62,NA,NA,NA,2,62
6,1002888,1,NA,NA,NA,68,NA,NA,NA,2,68


   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  40.00   61.00   65.00   63.83   68.00   81.00 

In [30]:
### Extract age for Control (3393)
aid_age_control <- hearing_aid %>% 
  filter(hearing_aid_cat == "0") %>%
  select(IID, f.3393.0.0_recode, f.3393.1.0_recode, f.3393.2.0_recode, f.3393.3.0_recode, f.21003.0.0, f.21003.1.0, f.21003.2.0, f.21003.3.0)
head(aid_age_control)

,IID,f.3393.0.0_recode,f.3393.1.0_recode,f.3393.2.0_recode,f.3393.3.0_recode,f.21003.0.0,f.21003.1.0,f.21003.2.0,f.21003.3.0
,<int>,<fct>,<fct>,<fct>,<fct>,<int>,<int>,<int>,<int>
1,1000019,0,NA,NA,NA,47,NA,NA,NA
2,1000022,0,NA,NA,NA,53,NA,NA,NA
3,1000035,0,NA,NA,NA,63,NA,NA,NA
4,1000046,NA,NA,0,NA,62,NA,73,NA
5,1000054,0,NA,NA,NA,65,NA,NA,NA
6,1000063,0,NA,NA,NA,43,NA,NA,NA


In [31]:
# Get the # of column where last replied No:
aid_age_control$visit_idx = apply(aid_age_control, 1, function(x) unlist(which(x == '0')))

# Define offset:
# offset: refers to the # of columns between the first age column (i.e.f.21003.0.0) and the first recode column (i.e.f.3393.0.0_recode)
offset = which(colnames(aid_age_control) == 'f.21003.0.0') - which(colnames(aid_age_control) == 'f.3393.0.0_recode')

# Define the function to extract the last time they said no for control

f=get_age_func <- function(x) {
  visit_index=x[which(colnames(aid_age_control)=="visit_idx")]
  index=max(unlist(visit_index))+offset
  age=x[index]
  age=unlist(age)
  return(age)
}

# Get the final age for controls
aid_age_control$age_final_aid = apply(aid_age_control, 1, f)

# Show first 6 rows
head(aid_age_control)
summary(aid_age_control$age_final_aid)

,IID,f.3393.0.0_recode,f.3393.1.0_recode,f.3393.2.0_recode,f.3393.3.0_recode,f.21003.0.0,f.21003.1.0,f.21003.2.0,f.21003.3.0,visit_idx,age_final_aid
,<int>,<fct>,<fct>,<fct>,<fct>,<int>,<int>,<int>,<int>,<list>,<int>
1,1000019,0,NA,NA,NA,47,NA,NA,NA,2,47
2,1000022,0,NA,NA,NA,53,NA,NA,NA,2,53
3,1000035,0,NA,NA,NA,63,NA,NA,NA,2,63
4,1000046,NA,NA,0,NA,62,NA,73,NA,4,73
5,1000054,0,NA,NA,NA,65,NA,NA,NA,2,65
6,1000063,0,NA,NA,NA,43,NA,NA,NA,2,43


   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
   39.0    53.0    60.0    58.6    65.0    82.0 

In [32]:
# Merge age_cases and age_controls
hearing_aid_age <- rbind(aid_age_cases, aid_age_control) 
dim(hearing_aid_age)
head(hearing_aid_age)

[1] 230411     11

,IID,f.3393.0.0_recode,f.3393.1.0_recode,f.3393.2.0_recode,f.3393.3.0_recode,f.21003.0.0,f.21003.1.0,f.21003.2.0,f.21003.3.0,visit_idx,age_final_aid
,<int>,<fct>,<fct>,<fct>,<fct>,<int>,<int>,<int>,<int>,<list>,<int>
1,1000112,NA,NA,1,NA,58,NA,68,NA,4,68
2,1001067,1,NA,NA,NA,50,NA,NA,NA,2,50
3,1001384,1,NA,NA,NA,61,NA,NA,NA,2,61
4,1001459,1,NA,NA,NA,64,NA,NA,NA,2,64
5,1002548,1,NA,NA,NA,62,NA,NA,NA,2,62
6,1002888,1,NA,NA,NA,68,NA,NA,NA,2,68


In [33]:
hearing_cat_age = merge(x = hearing_aid ,y = hearing_aid_age [,c("IID","age_final_aid")],by="IID", all.x=TRUE)
head(hearing_cat_age)

,IID,FID,f.22001.0.0,f.21003.0.0,f.21003.1.0,f.21003.2.0,f.21003.3.0,f.3393.0.0_recode,f.3393.1.0_recode,f.3393.2.0_recode,f.3393.3.0_recode,cases,hearing_aid_cat,age_final_aid
,<int>,<int>,<fct>,<int>,<int>,<int>,<int>,<fct>,<fct>,<fct>,<fct>,<lgl>,<int>,<int>
1,1000019,1000019,Female,47,NA,NA,NA,0,NA,NA,NA,FALSE,0,47
2,1000022,1000022,Male,53,NA,NA,NA,0,NA,NA,NA,FALSE,0,53
3,1000035,1000035,Male,63,NA,NA,NA,0,NA,NA,NA,FALSE,0,63
4,1000046,1000046,Female,62,NA,73,NA,NA,NA,0,NA,FALSE,0,73
5,1000054,1000054,Female,65,NA,NA,NA,0,NA,NA,NA,FALSE,0,65
6,1000063,1000063,Male,43,NA,NA,NA,0,NA,NA,NA,FALSE,0,43


In [34]:
#Merge with complete database keep the all the rows from original db
df_3393 = merge(x = df.final.imp, y = hearing_cat_age [,c("IID","hearing_aid_cat","age_final_aid")],by="IID", all.x=TRUE)
dim(df_3393)
head(df_3393)

[1] 354347   1089

,IID,FID,ignore1,ignore2,ignore3,ignore4,f.31.0.0,f.34.0.0,f.53.0.0,f.53.1.0,⋯,loud_music,noise_imp,music_imp,sex,f.3393.0.0_recode,f.3393.1.0_recode,f.3393.2.0_recode,f.3393.3.0_recode,hearing_aid_cat,age_final_aid
,<int>,<int>,<int>,<int>,<int>,<int>,<fct>,<int>,<fct>,<fct>,⋯,<int>,<int>,<int>,<fct>,<fct>,<fct>,<fct>,<fct>,<int>,<int>
1,1000019,1000019,0,0,2,-9,Female,1960,2008-01-24,NA,⋯,NA,NA,NA,1,0,NA,NA,NA,0,47
2,1000022,1000022,0,0,1,-9,Male,1954,2008-01-22,NA,⋯,NA,NA,NA,0,0,NA,NA,NA,0,53
3,1000035,1000035,0,0,1,-9,Male,1944,2007-11-08,NA,⋯,NA,NA,NA,0,0,NA,NA,NA,0,63
4,1000046,1000046,0,0,2,-9,Female,1946,2008-12-01,NA,⋯,0,0,0,1,NA,NA,0,NA,0,73
5,1000054,1000054,0,0,2,-9,Female,1942,2007-11-23,NA,⋯,NA,NA,NA,1,0,NA,NA,NA,0,65
6,1000063,1000063,0,0,1,-9,Male,1967,2010-06-26,NA,⋯,0,0,0,0,0,NA,NA,NA,0,43


## Export file for hearing aid analysis f.3393

In [35]:
f.3393_pheno <- df_3393 %>%
    filter(!(is.na(hearing_aid_cat))) %>%
    select(FID, IID, age_final_aid, sex, hearing_aid_cat)
head(f.3393_pheno)
dim(f.3393_pheno)

,FID,IID,age_final_aid,sex,hearing_aid_cat
,<int>,<int>,<int>,<fct>,<int>
1,1000019,1000019,47,1,0
2,1000022,1000022,53,0,0
3,1000035,1000035,63,0,0
4,1000046,1000046,73,1,0
5,1000054,1000054,65,1,0
6,1000063,1000063,43,0,0


[1] 230411      5

In [36]:
write.table(f.3393_pheno, '200828_UKBB_Hearing_aid_f3393', quote = FALSE, row.names = FALSE)

## Hearing difficulty/problems (2247)
### step1: classify cases and controls

In [37]:
table(df.final.imp$f.2247.0.0)


         Do not know I am completely deaf                   No 
               13469                   56               253610 
Prefer not to answer                  Yes 
                 170                86713 

In [38]:
# Recode function:
recode<-function(df,column_name){
  new_names<-c()
  for (i in column_name){
    new_column_name<-paste0(i,"_recode")
    new_names<-c(new_names,new_column_name)
    df[,new_column_name] <- revalue(df[,i], c("No"= 0, 
                                            "Yes" =1,
                                            "Prefer not to answer"= NA,
                                             "Do not know"=9,
                                             "I am completely deaf"=NA))
  }
  return (list(df=df,new_column_names=new_names))
}

# columns needs to be recoded:
column_name<-c("f.2247.0.0","f.2247.1.0","f.2247.2.0","f.2247.3.0")

# get a new data.frame with recoded columns added:
df.final.imp<-recode(df=df.final.imp,column_name)$df

# get names of recoded columns:
new_column_names<-recode(df=df.final.imp,column_name)$new_column_names

# show recode summary:
for (i in new_column_names)
{cat(i,"summary:");print(table(df.final.imp[,i]));cat("\n")}

The following `from` values were not present in `x`: I am completely deaf

The following `from` values were not present in `x`: I am completely deaf



f.2247.0.0_recode summary:
     9      0      1 
 13469 253610  86713 

f.2247.1.0_recode summary:
   9    0    1 
 785 9709 4861 

f.2247.2.0_recode summary:
    9     0     1 
 1921 21757 12038 

f.2247.3.0_recode summary:
   9    0    1 
 128 1474  808 



## f.2247 Get possible answers

In [39]:
# Extract subset of data only with the recode columns of tinnitus
data_sub <- df.final.imp %>%
  select("IID",all_of(new_column_names)) 

# Function to extract all the available answers for 4 visits
# and put them in one string as "0000", "111", "991", etc

f2<-function(x){
  visit<-''
  for (i in 2:5){ # do not take the first column (i.e. IID)
    if (!is.na(x[i]))
    {visit<-paste0(visit,x[i])}
  }
  if(is.null(visit)){visit=NA}
  else{visit=visit}
  return (visit)
}

# Apply the above function and remove NAs
data_sub$visit<-apply(data_sub, 1, f2)

names(table(data_sub$visit))

[1] ""     "0"    "00"   "000"  "0000" "0001" "0009" "001"  "0010" "0011"
[11] "0019" "009"  "0090" "0091" "0099" "01"   "010"  "0100" "0101" "011" 
[21] "0110" "0111" "019"  "0190" "09"   "090"  "0900" "0901" "0909" "091" 
[31] "0911" "099"  "0990" "0991" "0999" "1"    "10"   "100"  "1000" "1001"
[41] "101"  "1010" "1011" "1019" "109"  "1090" "1099" "11"   "110"  "1100"
[51] "1101" "111"  "1111" "1119" "119"  "1190" "19"   "190"  "1900" "191" 
[61] "1910" "1911" "199"  "9"    "90"   "900"  "9000" "9001" "9009" "901" 
[71] "9011" "909"  "9090" "9099" "91"   "910"  "911"  "9111" "919"  "99"  
[81] "990"  "991"  "999"  "9990"

In [40]:
table(data_sub$visit)
all_pattern<-unique(data_sub$visit)


            0     00    000   0000   0001   0009    001   0010   0011   0019 
   214 221437  20495   3918    222     18     11    460     11     22      2 
   009   0090   0091   0099     01    010   0100   0101    011   0110   0111 
   160      5      2      2   4607    174      5      3    523      2     23 
   019   0190     09    090   0900   0901   0909    091   0911    099   0990 
    33      1   1375    128      4      3      2     75      1     70      1 
  0991   0999      1     10    100   1000   1001    101   1010   1011   1019 
     3      3  76130   1313    153      5      2    105      1      5      1 
   109   1090   1099     11    110   1100   1101    111   1111   1119    119 
    13      4      1   7190    108      2      3   1294     66      1     34 
  1190     19    190   1900    191   1910   1911    199      9     90    900 
     1    344     19      1     24      1      2     15  11928    509     83 
  9000   9001   9009    901   9011    909   9090   9099     91 

## f.2247 Get all the consistent cases

In [41]:
all_no<-all_pattern[which(grepl("0",all_pattern) & !grepl("9",all_pattern) & !grepl("1",all_pattern))]
all_yes<-all_pattern[which(!grepl("0",all_pattern) & !grepl("9",all_pattern) & grepl("1",all_pattern))]
all_do_not_know<-all_pattern[which(!grepl("0",all_pattern) & grepl("9",all_pattern) & !grepl("1",all_pattern))]
table(data_sub[data_sub$visit%in%union(union(all_no,all_yes),all_do_not_know),"visit"])


     0     00    000   0000      1     11    111   1111      9     99    999 
221437  20495   3918    222  76130   7190   1294     66  11928    233     28 

In [42]:
consistent_1<-c(all_no,all_yes,all_do_not_know)

# might be inconsistent cases:
do_not_know_no<-all_pattern[which(grepl("0",all_pattern) & grepl("9",all_pattern) & !grepl("1",all_pattern))]
table(data_sub[data_sub$visit%in%do_not_know_no,"visit"])


0009  009 0090 0099   09  090 0900 0909  099 0990 0999   90  900 9000 9009  909 
  11  160    5    2 1375  128    4    2   70    1    3  509   83    4    1   23 
9090 9099  990 9990 
   1    1   27    1 

In [43]:
do_not_know_yes<-all_pattern[which(!grepl("0",all_pattern) & grepl("9",all_pattern) & grepl("1",all_pattern))]
table(data_sub[data_sub$visit%in%do_not_know_yes,"visit"])


1119  119   19  191 1911  199   91  911 9111  919  991 
   1   34  344   24    2   15  523   82    1    2   20 

In [44]:
yes_no<-all_pattern[which(grepl("0",all_pattern) & !grepl("9",all_pattern) & grepl("1",all_pattern))]
table(data_sub[data_sub$visit%in%yes_no,"visit"])


0001  001 0010 0011   01  010 0100 0101  011 0110 0111   10  100 1000 1001  101 
  18  460   11   22 4607  174    5    3  523    2   23 1313  153    5    2  105 
1010 1011  110 1100 1101 
   1    5  108    2    3 

In [45]:
with_all_three<-all_pattern[which(grepl("0",all_pattern) & grepl("9",all_pattern) & grepl("1",all_pattern))]
table(data_sub[data_sub$visit%in%with_all_three,"visit"])


0019 0091  019 0190 0901  091 0911 0991 1019  109 1090 1099 1190  190 1900 1910 
   2    2   33    1    3   75    1    3    1   13    4    1    1   19    1    1 
9001  901 9011  910 
   1   15    1   10 

In [46]:
might_inconsistent<-c(do_not_know_no,do_not_know_yes,yes_no,with_all_three)

## f.2247 Remove inconsistent cases

In [47]:
# get all the inconsistent cases:
exceptions<-c("91","911","9111","991","0001","001","0011","01","011","0111", "0091", "091","0911","0991","9001","901","9011")
inconsistent<-setdiff(might_inconsistent,exceptions)

cat("There are",length(which(is.empty(data_sub$visit))),"NAs, removed")

There are 214 NAs, removed

In [48]:
cat("There are",length(which(data_sub$visit%in%inconsistent)),"inconsistent cases, removed")

There are 4815 inconsistent cases, removed

In [49]:
# removed those NAs, consistent for do not know - 9 and inconsistent cases: 
IID_hearing_diff<-data_sub %>%
  filter(!is.empty(visit)) %>%
  filter((!visit%in%inconsistent)) %>%
  filter((!visit%in%all_do_not_know)) %>%
  select(IID)

dim(IID_hearing_diff)

[1] 337129      1

In [50]:
IID_hearing_diff<-IID_hearing_diff[,1]
cat("After removing all the NAs and inconsistent cases, there are",length(IID_hearing_diff),"cases left")

After removing all the NAs and inconsistent cases, there are 337129 cases left

In [51]:
# get the dataset after cleansing: 
df.hearing.diff<-df.final.imp %>%
  filter(IID%in%IID_hearing_diff)

dim(df.hearing.diff)

[1] 337129   1091

## f.2247 Step 1 classify cases and controls

In [52]:
hearing_diff <- df.hearing.diff %>% 
  select(IID,FID,f.22001.0.0, f.21003.0.0,f.21003.1.0,f.21003.2.0,f.21003.3.0,f.2247.0.0_recode,f.2247.1.0_recode,f.2247.2.0_recode,f.2247.3.0_recode)
head(hearing_diff)

,IID,FID,f.22001.0.0,f.21003.0.0,f.21003.1.0,f.21003.2.0,f.21003.3.0,f.2247.0.0_recode,f.2247.1.0_recode,f.2247.2.0_recode,f.2247.3.0_recode
,<int>,<int>,<fct>,<int>,<int>,<int>,<int>,<fct>,<fct>,<fct>,<fct>
1,1000019,1000019,Female,47,NA,NA,NA,1,NA,NA,NA
2,1000022,1000022,Male,53,NA,NA,NA,1,NA,NA,NA
3,1000035,1000035,Male,63,NA,NA,NA,0,NA,NA,NA
4,1000046,1000046,Female,62,NA,73,NA,0,NA,0,NA
5,1000054,1000054,Female,65,NA,NA,NA,0,NA,NA,NA
6,1000063,1000063,Male,43,NA,NA,NA,0,NA,NA,NA


In [53]:
data_sub <- select(hearing_diff,starts_with("f.2247"))

# Function to define cases
f<-function(x){
  visit<-c()
  for (i in 1:4){
    if (!is.na(x[i]))
    {visit<-c(visit,x[i])}
  }
  if ("1" %in% visit){result= TRUE}
  else{result=FALSE}
  return (result)
}

# Apply the above function
hearing_diff$cases<-apply(data_sub, 1, f)
head(hearing_diff,10)

,IID,FID,f.22001.0.0,f.21003.0.0,f.21003.1.0,f.21003.2.0,f.21003.3.0,f.2247.0.0_recode,f.2247.1.0_recode,f.2247.2.0_recode,f.2247.3.0_recode,cases
,<int>,<int>,<fct>,<int>,<int>,<int>,<int>,<fct>,<fct>,<fct>,<fct>,<lgl>
1,1000019,1000019,Female,47,NA,NA,NA,1,NA,NA,NA,TRUE
2,1000022,1000022,Male,53,NA,NA,NA,1,NA,NA,NA,TRUE
3,1000035,1000035,Male,63,NA,NA,NA,0,NA,NA,NA,FALSE
4,1000046,1000046,Female,62,NA,73,NA,0,NA,0,NA,FALSE
5,1000054,1000054,Female,65,NA,NA,NA,0,NA,NA,NA,FALSE
6,1000063,1000063,Male,43,NA,NA,NA,0,NA,NA,NA,FALSE
7,1000078,1000078,Female,52,57,60,NA,0,0,0,NA,FALSE
8,1000112,1000112,Male,58,NA,68,NA,9,NA,1,NA,TRUE
9,1000129,1000129,Male,62,NA,NA,NA,0,NA,NA,NA,FALSE


In [54]:
hearing_diff_cases <- hearing_diff %>% 
  filter(cases == TRUE)
head(hearing_diff_cases)
cat(dim(hearing_diff_cases)[1], "individuals are cases for f.2247")

,IID,FID,f.22001.0.0,f.21003.0.0,f.21003.1.0,f.21003.2.0,f.21003.3.0,f.2247.0.0_recode,f.2247.1.0_recode,f.2247.2.0_recode,f.2247.3.0_recode,cases
,<int>,<int>,<fct>,<int>,<int>,<int>,<int>,<fct>,<fct>,<fct>,<fct>,<lgl>
1,1000019,1000019,Female,47,NA,NA,NA,1,NA,NA,NA,TRUE
2,1000022,1000022,Male,53,NA,NA,NA,1,NA,NA,NA,TRUE
3,1000112,1000112,Male,58,NA,68,NA,9,NA,1,NA,TRUE
4,1000170,1000170,Female,51,NA,NA,NA,1,NA,NA,NA,TRUE
5,1000198,1000198,Female,41,NA,NA,NA,1,NA,NA,NA,TRUE
6,1000203,1000203,Female,49,NA,NA,NA,1,NA,NA,NA,TRUE


91057 individuals are cases for f.2247

In [55]:
hearing_diff_controls <- hearing_diff %>% 
  filter(cases == FALSE)
head(hearing_diff_controls)
cat(dim(hearing_diff_controls)[1], "individuals are controls for f.2247")

,IID,FID,f.22001.0.0,f.21003.0.0,f.21003.1.0,f.21003.2.0,f.21003.3.0,f.2247.0.0_recode,f.2247.1.0_recode,f.2247.2.0_recode,f.2247.3.0_recode,cases
,<int>,<int>,<fct>,<int>,<int>,<int>,<int>,<fct>,<fct>,<fct>,<fct>,<lgl>
1,1000035,1000035,Male,63,NA,NA,NA,0,NA,NA,NA,FALSE
2,1000046,1000046,Female,62,NA,73,NA,0,NA,0,NA,FALSE
3,1000054,1000054,Female,65,NA,NA,NA,0,NA,NA,NA,FALSE
4,1000063,1000063,Male,43,NA,NA,NA,0,NA,NA,NA,FALSE
5,1000078,1000078,Female,52,57,60,NA,0,0,0,NA,FALSE
6,1000129,1000129,Male,62,NA,NA,NA,0,NA,NA,NA,FALSE


246072 individuals are controls for f.2247

In [56]:
dim(hearing_diff)[1]-246072 - 91057

[1] 0

## f.2247 Recode cases=1 and controls=0

In [57]:
hearing_diff$hearing_diff_cat <- as.integer(as.logical(hearing_diff$cases))
head(hearing_diff, 10)
# Get the number of NAs
length(which(is.na(hearing_diff$hearing_dif_cat)))

,IID,FID,f.22001.0.0,f.21003.0.0,f.21003.1.0,f.21003.2.0,f.21003.3.0,f.2247.0.0_recode,f.2247.1.0_recode,f.2247.2.0_recode,f.2247.3.0_recode,cases,hearing_diff_cat
,<int>,<int>,<fct>,<int>,<int>,<int>,<int>,<fct>,<fct>,<fct>,<fct>,<lgl>,<int>
1,1000019,1000019,Female,47,NA,NA,NA,1,NA,NA,NA,TRUE,1
2,1000022,1000022,Male,53,NA,NA,NA,1,NA,NA,NA,TRUE,1
3,1000035,1000035,Male,63,NA,NA,NA,0,NA,NA,NA,FALSE,0
4,1000046,1000046,Female,62,NA,73,NA,0,NA,0,NA,FALSE,0
5,1000054,1000054,Female,65,NA,NA,NA,0,NA,NA,NA,FALSE,0
6,1000063,1000063,Male,43,NA,NA,NA,0,NA,NA,NA,FALSE,0
7,1000078,1000078,Female,52,57,60,NA,0,0,0,NA,FALSE,0
8,1000112,1000112,Male,58,NA,68,NA,9,NA,1,NA,TRUE,1
9,1000129,1000129,Male,62,NA,NA,NA,0,NA,NA,NA,FALSE,0


[1] 0

## f.2247 Step 2: get the ages for hearing difficulty

In [58]:
# Get the subset data of cases
diff_age_cases = hearing_diff %>% 
  filter(hearing_diff_cat=="1")  %>%
  select(IID, f.2247.0.0_recode, f.2247.1.0_recode, f.2247.2.0_recode, f.2247.3.0_recode, f.21003.0.0, f.21003.1.0, f.21003.2.0, f.21003.3.0)
head(diff_age_cases,12)

,IID,f.2247.0.0_recode,f.2247.1.0_recode,f.2247.2.0_recode,f.2247.3.0_recode,f.21003.0.0,f.21003.1.0,f.21003.2.0,f.21003.3.0
,<int>,<fct>,<fct>,<fct>,<fct>,<int>,<int>,<int>,<int>
1,1000019,1,NA,NA,NA,47,NA,NA,NA
2,1000022,1,NA,NA,NA,53,NA,NA,NA
3,1000112,9,NA,1,NA,58,NA,68,NA
4,1000170,1,NA,NA,NA,51,NA,NA,NA
5,1000198,1,NA,NA,NA,41,NA,NA,NA
6,1000203,1,NA,NA,NA,49,NA,NA,NA
7,1000494,1,NA,NA,NA,61,NA,NA,NA
8,1000549,1,NA,NA,NA,63,NA,NA,NA
9,1000728,1,NA,NA,NA,61,NA,NA,NA


In [59]:
# Get the # of column where first replied Yes:
diff_age_cases$visit_idx = apply(diff_age_cases, 1, function(x) unlist(which(x == '1')))
head(diff_age_cases)

,IID,f.2247.0.0_recode,f.2247.1.0_recode,f.2247.2.0_recode,f.2247.3.0_recode,f.21003.0.0,f.21003.1.0,f.21003.2.0,f.21003.3.0,visit_idx
,<int>,<fct>,<fct>,<fct>,<fct>,<int>,<int>,<int>,<int>,<list>
1,1000019,1,NA,NA,NA,47,NA,NA,NA,2
2,1000022,1,NA,NA,NA,53,NA,NA,NA,2
3,1000112,9,NA,1,NA,58,NA,68,NA,4
4,1000170,1,NA,NA,NA,51,NA,NA,NA,2
5,1000198,1,NA,NA,NA,41,NA,NA,NA,2
6,1000203,1,NA,NA,NA,49,NA,NA,NA,2


In [60]:
# Define offset:
# offset: refers to the # of columns between the first age column (i.e.f.21003.0.0) and the first recode column (i.e.f.22247.0.0_recode)
offset = which(colnames(diff_age_cases) == 'f.21003.0.0') - which(colnames(diff_age_cases) == 'f.2247.0.0_recode')

# Define the function to extract the first time they said yes for cases 
f=get_age_func <- function(x) {
  visit_index=x[which(colnames(diff_age_cases)=="visit_idx")]
  index=min(unlist(visit_index))+offset
  age=x[index]
  final_age=unlist(age)
  if(is.null(final_age))
  {final_age<-NA}
  return(final_age)
}

# Get the final age for cases
diff_age_cases$age_final_diff = apply(diff_age_cases, 1, f)

# Show first 6 rows
head(diff_age_cases)
summary(diff_age_cases$age_final_diff)

,IID,f.2247.0.0_recode,f.2247.1.0_recode,f.2247.2.0_recode,f.2247.3.0_recode,f.21003.0.0,f.21003.1.0,f.21003.2.0,f.21003.3.0,visit_idx,age_final_diff
,<int>,<fct>,<fct>,<fct>,<fct>,<int>,<int>,<int>,<int>,<list>,<int>
1,1000019,1,NA,NA,NA,47,NA,NA,NA,2,47
2,1000022,1,NA,NA,NA,53,NA,NA,NA,2,53
3,1000112,9,NA,1,NA,58,NA,68,NA,4,68
4,1000170,1,NA,NA,NA,51,NA,NA,NA,2,51
5,1000198,1,NA,NA,NA,41,NA,NA,NA,2,41
6,1000203,1,NA,NA,NA,49,NA,NA,NA,2,49


   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  39.00   55.00   61.00   59.48   65.00   82.00 

In [61]:
### Extract age for Control (2247)
diff_age_control <- hearing_diff %>% 
  filter(hearing_diff_cat == "0") %>%
  select(IID, f.2247.0.0_recode, f.2247.1.0_recode, f.2247.2.0_recode, f.2247.3.0_recode, f.21003.0.0, f.21003.1.0, f.21003.2.0, f.21003.3.0)
head(diff_age_control)

,IID,f.2247.0.0_recode,f.2247.1.0_recode,f.2247.2.0_recode,f.2247.3.0_recode,f.21003.0.0,f.21003.1.0,f.21003.2.0,f.21003.3.0
,<int>,<fct>,<fct>,<fct>,<fct>,<int>,<int>,<int>,<int>
1,1000035,0,NA,NA,NA,63,NA,NA,NA
2,1000046,0,NA,0,NA,62,NA,73,NA
3,1000054,0,NA,NA,NA,65,NA,NA,NA
4,1000063,0,NA,NA,NA,43,NA,NA,NA
5,1000078,0,0,0,NA,52,57,60,NA
6,1000129,0,NA,NA,NA,62,NA,NA,NA


In [62]:
# Get the # of column where last replied No:
diff_age_control$visit_idx = apply(diff_age_control, 1, function(x) unlist(which(x == '0')))

# Define offset:
# offset: refers to the # of columns between the first age column (i.e.f.21003.0.0) and the first recode column (i.e.f.3393.0.0_recode)
offset = which(colnames(diff_age_control) == 'f.21003.0.0') - which(colnames(diff_age_control) == 'f.2247.0.0_recode')

# Define the function to extract the last time they said no for control

f=get_age_func <- function(x) {
  visit_index=x[which(colnames(diff_age_control)=="visit_idx")]
  index=max(unlist(visit_index))+offset
  age=x[index]
  age=unlist(age)
  return(age)
}

# Get the final age for controls
diff_age_control$age_final_diff = apply(diff_age_control, 1, f)

# Show first 6 rows
head(diff_age_control)
summary(diff_age_control$age_final_diff)

,IID,f.2247.0.0_recode,f.2247.1.0_recode,f.2247.2.0_recode,f.2247.3.0_recode,f.21003.0.0,f.21003.1.0,f.21003.2.0,f.21003.3.0,visit_idx,age_final_diff
,<int>,<fct>,<fct>,<fct>,<fct>,<int>,<int>,<int>,<int>,<list>,<int>
1,1000035,0,NA,NA,NA,63,NA,NA,NA,2,63
2,1000046,0,NA,0,NA,62,NA,73,NA,"2, 4",73
3,1000054,0,NA,NA,NA,65,NA,NA,NA,2,65
4,1000063,0,NA,NA,NA,43,NA,NA,NA,2,43
5,1000078,0,0,0,NA,52,57,60,NA,"2, 3, 4",60
6,1000129,0,NA,NA,NA,62,NA,NA,NA,2,62


   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  39.00   50.00   58.00   56.98   64.00   82.00 

In [63]:
# Merge age_cases and age_controls
hearing_diff_age <- rbind(diff_age_cases, diff_age_control) 
dim(hearing_diff_age)
head(hearing_diff_age)

[1] 337129     11

,IID,f.2247.0.0_recode,f.2247.1.0_recode,f.2247.2.0_recode,f.2247.3.0_recode,f.21003.0.0,f.21003.1.0,f.21003.2.0,f.21003.3.0,visit_idx,age_final_diff
,<int>,<fct>,<fct>,<fct>,<fct>,<int>,<int>,<int>,<int>,<list>,<int>
1,1000019,1,NA,NA,NA,47,NA,NA,NA,2,47
2,1000022,1,NA,NA,NA,53,NA,NA,NA,2,53
3,1000112,9,NA,1,NA,58,NA,68,NA,4,68
4,1000170,1,NA,NA,NA,51,NA,NA,NA,2,51
5,1000198,1,NA,NA,NA,41,NA,NA,NA,2,41
6,1000203,1,NA,NA,NA,49,NA,NA,NA,2,49


In [64]:
hearing_diff_age = merge(x = hearing_diff ,y = hearing_diff_age [,c("IID","age_final_diff")],by="IID", all.x=TRUE)
head(hearing_diff_age)
dim(hearing_diff_age)

,IID,FID,f.22001.0.0,f.21003.0.0,f.21003.1.0,f.21003.2.0,f.21003.3.0,f.2247.0.0_recode,f.2247.1.0_recode,f.2247.2.0_recode,f.2247.3.0_recode,cases,hearing_diff_cat,age_final_diff
,<int>,<int>,<fct>,<int>,<int>,<int>,<int>,<fct>,<fct>,<fct>,<fct>,<lgl>,<int>,<int>
1,1000019,1000019,Female,47,NA,NA,NA,1,NA,NA,NA,TRUE,1,47
2,1000022,1000022,Male,53,NA,NA,NA,1,NA,NA,NA,TRUE,1,53
3,1000035,1000035,Male,63,NA,NA,NA,0,NA,NA,NA,FALSE,0,63
4,1000046,1000046,Female,62,NA,73,NA,0,NA,0,NA,FALSE,0,73
5,1000054,1000054,Female,65,NA,NA,NA,0,NA,NA,NA,FALSE,0,65
6,1000063,1000063,Male,43,NA,NA,NA,0,NA,NA,NA,FALSE,0,43


[1] 337129     14

## Merge dataset from f.3393 and f.2247

In [65]:
#Merge with complete database keep the all the rows from original db
df_3393_2247 = merge(x = df_3393, y = hearing_diff_age [,c("IID","hearing_diff_cat","age_final_diff")],by="IID", all.x=TRUE)
dim(df_3393_2247)
head(df_3393_2247)

[1] 354347   1091

,IID,FID,ignore1,ignore2,ignore3,ignore4,f.31.0.0,f.34.0.0,f.53.0.0,f.53.1.0,⋯,music_imp,sex,f.3393.0.0_recode,f.3393.1.0_recode,f.3393.2.0_recode,f.3393.3.0_recode,hearing_aid_cat,age_final_aid,hearing_diff_cat,age_final_diff
,<int>,<int>,<int>,<int>,<int>,<int>,<fct>,<int>,<fct>,<fct>,⋯,<int>,<fct>,<fct>,<fct>,<fct>,<fct>,<int>,<int>,<int>,<int>
1,1000019,1000019,0,0,2,-9,Female,1960,2008-01-24,NA,⋯,NA,1,0,NA,NA,NA,0,47,1,47
2,1000022,1000022,0,0,1,-9,Male,1954,2008-01-22,NA,⋯,NA,0,0,NA,NA,NA,0,53,1,53
3,1000035,1000035,0,0,1,-9,Male,1944,2007-11-08,NA,⋯,NA,0,0,NA,NA,NA,0,63,0,63
4,1000046,1000046,0,0,2,-9,Female,1946,2008-12-01,NA,⋯,0,1,NA,NA,0,NA,0,73,0,73
5,1000054,1000054,0,0,2,-9,Female,1942,2007-11-23,NA,⋯,NA,1,0,NA,NA,NA,0,65,0,65
6,1000063,1000063,0,0,1,-9,Male,1967,2010-06-26,NA,⋯,0,0,0,NA,NA,NA,0,43,0,43


## Reclassify individuals wearing hearing aid f.3393 but saying they don't have hearing difficulty f.2247

In [66]:
#reclassify these 85 individulas as cases for 2247
df_3393_2247$hearing_diff_cat_new <- as.factor(df_3393_2247$hearing_diff_cat)
df_3393_2247$hearing_diff_cat_new <- revalue(df_3393_2247$hearing_diff_cat_new, c('0' = "control", '1' = 'case' ))
df_3393_2247$hearing_aid_cat_new <- as.factor(df_3393_2247$hearing_aid_cat)
df_3393_2247$hearing_aid_cat_new <- revalue(df_3393_2247$hearing_aid_cat_new, c('0' = "control", '1' = 'case' ))
#hearing_diff_new <- df_3393_2247 %>% 
 # mutate(hearing_diff_cat_new =ifelse(IID%in%inconsistent_aid_diff,1,hearing_diff_cat_new))
#head(hearing_diff_new)

In [67]:
246072 + 91057 # number of controls and cases

[1] 337129

In [68]:
length(which(df_3393_2247$hearing_diff_cat_new == 'control' & df_3393_2247$hearing_aid_cat_new == 'case'))

[1] 85

In [69]:
#85 individuals state they have no hearing difficulty but they wear hearing aids
inconsistent_aid_diff <- df_3393_2247 %>%
    filter(hearing_diff_cat_new == 'control' & hearing_aid_cat_new == 'case') %>%
    select(IID)
nrow(inconsistent_aid_diff)
head(inconsistent_aid_diff)

[1] 85

,IID
,<int>
1,1011166
2,1085722
3,1156617
4,1178338
5,1200917
6,1210033


In [70]:
inconsistent_aid_diff<-unlist(inconsistent_aid_diff[,1])
cat("There are",length(inconsistent_aid_diff),"individuals that state having no hearing difficulty but wear hearing aid")

There are 85 individuals that state having no hearing difficulty but wear hearing aid

In [71]:
df_3393_2247$hearing_diff_new <- df_3393_2247$hearing_diff_cat_new
df_3393_2247$hearing_diff_new[df_3393_2247$IID %in% inconsistent_aid_diff] <- "case"

In [72]:
#91057 individuals plus 85 that wear hearing aid = 91142
check_merge <- df_3393_2247 %>% 
  select(hearing_diff_cat_new, hearing_aid_cat_new, hearing_diff_new) %>%
  filter(hearing_diff_new == "case")
head(check_merge)
nrow(check_merge)

,hearing_diff_cat_new,hearing_aid_cat_new,hearing_diff_new
,<fct>,<fct>,<fct>
1,case,control,case
2,case,control,case
3,case,case,case
4,case,control,case
5,case,control,case
6,case,control,case


[1] 91142

## f.2247 Verify the age for the new 85 cases

In [73]:
age_inconsistent <- df_3393_2247 %>%
    select(IID, age_final_aid, age_final_diff) %>%
    filter(IID %in% inconsistent_aid_diff)
head(age_inconsistent, 100)

,IID,age_final_aid,age_final_diff
,<int>,<int>,<int>
1,1011166,52,52
2,1085722,43,43
3,1156617,65,65
4,1178338,40,40
5,1200917,70,70
6,1210033,60,60
7,1362299,45,45
8,1541460,66,66
9,1546451,57,57


In [74]:
# There are not inconsistencies in the age of the 85 new cases therefore I can use the age_final_diff variable
all(age_inconsistent$age_final_aid == age_inconsistent$age_final_diff)

[1] TRUE

## Export file for hearing difficulty f.2247

In [75]:
df_3393_2247$hearing_diff_new <- revalue(df_3393_2247$hearing_diff_new, c('control' = "0", 'case' = '1' ))

In [76]:
f.2247_pheno <- df_3393_2247 %>%
    filter(!(is.na(hearing_diff_new))) %>%
    select(FID, IID, age_final_diff, sex, hearing_diff_new)
head(f.2247_pheno)
dim(f.2247_pheno)

,FID,IID,age_final_diff,sex,hearing_diff_new
,<int>,<int>,<int>,<fct>,<fct>
1,1000019,1000019,47,1,1
2,1000022,1000022,53,0,1
3,1000035,1000035,63,0,0
4,1000046,1000046,73,1,0
5,1000054,1000054,65,1,0
6,1000063,1000063,43,0,0


[1] 337129      5

In [77]:
write.table(f.2247_pheno, '200828_UKBB_Hearing_difficulty_f2247', quote = FALSE, row.names = FALSE)

## f.2257 Hearing difficulty/background noise (2257)
### step1: classify cases and controls

In [78]:
table(df.final.imp$f.2257.0.0)


         Do not know                   No Prefer not to answer 
                6494               218090                  208 
                 Yes 
              129498 

In [79]:
# Recode function:
recode<-function(df,column_name){
  new_names<-c()
  for (i in column_name){
    new_column_name<-paste0(i,"_recode")
    new_names<-c(new_names,new_column_name)
    df[,new_column_name] <- revalue(df[,i], c("No"= 0, 
                                            "Yes" =1,
                                            "Prefer not to answer"= NA,
                                             "Do not know"=9))
  }
  return (list(df=df,new_column_names=new_names))
}

# columns needs to be recoded:
column_name<-c("f.2257.0.0","f.2257.1.0","f.2257.2.0","f.2257.3.0")

# get a new data.frame with recoded columns added:
df.final.imp<-recode(df=df.final.imp,column_name)$df

# get names of recoded columns:
new_column_names<-recode(df=df.final.imp,column_name)$new_column_names

# show recode summary:
for (i in new_column_names)
{cat(i,"summary:");print(table(df.final.imp[,i]));cat("\n")}

f.2257.0.0_recode summary:
     9      0      1 
  6494 218090 129498 

f.2257.1.0_recode summary:
   9    0    1 
 311 8517 6528 

f.2257.2.0_recode summary:
    9     0     1 
  780 18972 15952 

f.2257.3.0_recode summary:
   9    0    1 
  52 1320 1037 



In [80]:
dim(df.final.imp)

[1] 354347   1095

## f.2257 Get patterns for all possible answers

In [81]:
# Extract subset of data only with the recode columns of tinnitus
data_sub <- df.final.imp %>%
  select("IID",all_of(new_column_names)) 

# Function to extract all the available answers for 4 visits
# and put them in one string as "0000", "111", "991", etc

f2<-function(x){
  visit<-''
  for (i in 2:5){ # do not take the first column (i.e. IID)
    if (!is.na(x[i]))
    {visit<-paste0(visit,x[i])}
  }
  if(is.null(visit)){visit=NA}
  else{visit=visit}
  return (visit)
}

# Apply the above function and remove NAs
data_sub$visit<-apply(data_sub, 1, f2)

names(table(data_sub$visit))

[1] ""     "0"    "00"   "000"  "0000" "0001" "0009" "001"  "0010" "0011"
[11] "0019" "009"  "0090" "0091" "01"   "010"  "0100" "0101" "011"  "0110"
[21] "0111" "019"  "09"   "090"  "0900" "0901" "091"  "0911" "099"  "1"   
[31] "10"   "100"  "1000" "1001" "101"  "1010" "1011" "109"  "11"   "110" 
[41] "1100" "1101" "111"  "1110" "1111" "1119" "119"  "19"   "190"  "1900"
[51] "191"  "199"  "9"    "90"   "900"  "9000" "9001" "901"  "9011" "909" 
[61] "91"   "910"  "911"  "9110" "9111" "919"  "9191" "99"   "990"  "991" 
[71] "999"  "9991" "9999"

In [82]:
table(data_sub$visit)
all_pattern<-unique(data_sub$visit)


            0     00    000   0000   0001   0009    001   0010   0011   0019 
   248 190353  16974   3225    181     24      2    533     10     24      1 
   009   0090   0091     01    010   0100   0101    011   0110   0111    019 
    67      5      2   5164    272      9      6    594      7     26     19 
    09    090   0900   0901    091   0911    099      1     10    100   1000 
   500     55      3      1     29      2      8 113147   2301    307      7 
  1001    101   1010   1011    109     11    110   1100   1101    111   1110 
     4    215      2      9     15  10906    176      9      6   2019      6 
  1111   1119    119     19    190   1900    191    199      9     90    900 
   101      1     26    210     10      2     24      5   5785    249     31 
  9000   9001    901   9011    909     91    910    911   9110   9111    919 
     2      1     12      1      7    249      7     29      1      2      6 
  9191     99    990    991    999   9991   9999 
     1     84

## f.2257 Get all the consitent cases

In [83]:
all_no<-all_pattern[which(grepl("0",all_pattern) & !grepl("9",all_pattern) & !grepl("1",all_pattern))]
all_yes<-all_pattern[which(!grepl("0",all_pattern) & !grepl("9",all_pattern) & grepl("1",all_pattern))]
all_do_not_know<-all_pattern[which(!grepl("0",all_pattern) & grepl("9",all_pattern) & !grepl("1",all_pattern))]
table(data_sub[data_sub$visit%in%union(union(all_no,all_yes),all_do_not_know),"visit"])


     0     00    000   0000      1     11    111   1111      9     99    999 
190353  16974   3225    181 113147  10906   2019    101   5785     84     11 
  9999 
     2 

In [84]:
consistent_1<-c(all_no,all_yes,all_do_not_know)

# might be inconsistent cases:
do_not_know_no<-all_pattern[which(grepl("0",all_pattern) & grepl("9",all_pattern) & !grepl("1",all_pattern))]
table(data_sub[data_sub$visit%in%do_not_know_no,"visit"])


0009  009 0090   09  090 0900  099   90  900 9000  909  990 
   2   67    5  500   55    3    8  249   31    2    7    8 

In [85]:
do_not_know_yes<-all_pattern[which(!grepl("0",all_pattern) & grepl("9",all_pattern) & grepl("1",all_pattern))]
table(data_sub[data_sub$visit%in%do_not_know_yes,"visit"])


1119  119   19  191  199   91  911 9111  919 9191  991 9991 
   1   26  210   24    5  249   29    2    6    1    6    1 

In [86]:
yes_no<-all_pattern[which(grepl("0",all_pattern) & !grepl("9",all_pattern) & grepl("1",all_pattern))]
table(data_sub[data_sub$visit%in%yes_no,"visit"])


0001  001 0010 0011   01  010 0100 0101  011 0110 0111   10  100 1000 1001  101 
  24  533   10   24 5164  272    9    6  594    7   26 2301  307    7    4  215 
1010 1011  110 1100 1101 1110 
   2    9  176    9    6    6 

In [87]:
with_all_three<-all_pattern[which(grepl("0",all_pattern) & grepl("9",all_pattern) & grepl("1",all_pattern))]
table(data_sub[data_sub$visit%in%with_all_three,"visit"])


0019 0091  019 0901  091 0911  109  190 1900 9001  901 9011  910 9110 
   1    2   19    1   29    2   15   10    2    1   12    1    7    1 

In [88]:
might_inconsistent<-c(do_not_know_no,do_not_know_yes,yes_no,with_all_three)

## f.2257 Remove all the inconsistent

In [89]:
# get all the inconsistent cases:
exceptions<-c("91","911","9111","991","9991","0001","001","0011","01","011","0111", "0091", "091","0911","9001","901","9011")
inconsistent<-setdiff(might_inconsistent,exceptions)

cat("There are",length(which(is.empty(data_sub$visit))),"NAs, removed")

There are 248 NAs, removed

In [90]:
cat("There are",length(which(data_sub$visit%in%inconsistent)),"inconsistent cases, removed")

There are 4612 inconsistent cases, removed

In [91]:
# removed those NAs, consistent for do not know - 9 and inconsistent cases: 
IID_hearing_noise<-data_sub %>%
  filter(!is.empty(visit)) %>%
  filter((!visit%in%inconsistent)) %>%
  filter((!visit%in%all_do_not_know)) %>%
  select(IID)

dim(IID_hearing_noise)

[1] 343605      1

In [92]:
IID_hearing_noise<-IID_hearing_noise[,1]
cat("After removing all the NAs and inconsistent cases, there are",length(IID_hearing_noise),"cases left")

After removing all the NAs and inconsistent cases, there are 343605 cases left

In [93]:
# get the dataset after cleansing: 
df.hearing.noise<-df.final.imp %>%
  filter(IID%in%IID_hearing_noise)

dim(df.hearing.noise)

[1] 343605   1095

## f.2257 Step 1 classify cases and controls

In [94]:
hearing_noise <- df.hearing.noise %>% 
  select(IID,FID,f.22001.0.0, f.21003.0.0,f.21003.1.0,f.21003.2.0,f.21003.3.0,f.2257.0.0_recode,f.2257.1.0_recode,f.2257.2.0_recode,f.2257.3.0_recode)
head(hearing_noise)

,IID,FID,f.22001.0.0,f.21003.0.0,f.21003.1.0,f.21003.2.0,f.21003.3.0,f.2257.0.0_recode,f.2257.1.0_recode,f.2257.2.0_recode,f.2257.3.0_recode
,<int>,<int>,<fct>,<int>,<int>,<int>,<int>,<fct>,<fct>,<fct>,<fct>
1,1000019,1000019,Female,47,NA,NA,NA,1,NA,NA,NA
2,1000022,1000022,Male,53,NA,NA,NA,1,NA,NA,NA
3,1000035,1000035,Male,63,NA,NA,NA,1,NA,NA,NA
4,1000046,1000046,Female,62,NA,73,NA,0,NA,1,NA
5,1000054,1000054,Female,65,NA,NA,NA,1,NA,NA,NA
6,1000063,1000063,Male,43,NA,NA,NA,0,NA,NA,NA


In [95]:
data_sub <- select(hearing_noise,starts_with("f.2257"))

# Function to define cases
f<-function(x){
  visit<-c()
  for (i in 1:4){
    if (!is.na(x[i]))
    {visit<-c(visit,x[i])}
  }
  if ("1" %in% visit){result= TRUE}
  else{result=FALSE}
  return (result)
}

# Apply the above function
hearing_noise$cases<-apply(data_sub, 1, f)
head(hearing_noise,10)

,IID,FID,f.22001.0.0,f.21003.0.0,f.21003.1.0,f.21003.2.0,f.21003.3.0,f.2257.0.0_recode,f.2257.1.0_recode,f.2257.2.0_recode,f.2257.3.0_recode,cases
,<int>,<int>,<fct>,<int>,<int>,<int>,<int>,<fct>,<fct>,<fct>,<fct>,<lgl>
1,1000019,1000019,Female,47,NA,NA,NA,1,NA,NA,NA,TRUE
2,1000022,1000022,Male,53,NA,NA,NA,1,NA,NA,NA,TRUE
3,1000035,1000035,Male,63,NA,NA,NA,1,NA,NA,NA,TRUE
4,1000046,1000046,Female,62,NA,73,NA,0,NA,1,NA,TRUE
5,1000054,1000054,Female,65,NA,NA,NA,1,NA,NA,NA,TRUE
6,1000063,1000063,Male,43,NA,NA,NA,0,NA,NA,NA,FALSE
7,1000078,1000078,Female,52,57,60,NA,0,0,0,NA,FALSE
8,1000105,1000105,Female,54,NA,NA,NA,1,NA,NA,NA,TRUE
9,1000112,1000112,Male,58,NA,68,NA,0,NA,1,NA,TRUE


In [96]:
hearing_noise_cases <- hearing_noise %>% 
  filter(cases == TRUE)
head(hearing_noise_cases)
cat(dim(hearing_noise_cases)[1], "individuals are cases for f.2257")

,IID,FID,f.22001.0.0,f.21003.0.0,f.21003.1.0,f.21003.2.0,f.21003.3.0,f.2257.0.0_recode,f.2257.1.0_recode,f.2257.2.0_recode,f.2257.3.0_recode,cases
,<int>,<int>,<fct>,<int>,<int>,<int>,<int>,<fct>,<fct>,<fct>,<fct>,<lgl>
1,1000019,1000019,Female,47,NA,NA,NA,1,NA,NA,NA,TRUE
2,1000022,1000022,Male,53,NA,NA,NA,1,NA,NA,NA,TRUE
3,1000035,1000035,Male,63,NA,NA,NA,1,NA,NA,NA,TRUE
4,1000046,1000046,Female,62,NA,73,NA,0,NA,1,NA,TRUE
5,1000054,1000054,Female,65,NA,NA,NA,1,NA,NA,NA,TRUE
6,1000105,1000105,Female,54,NA,NA,NA,1,NA,NA,NA,TRUE


132872 individuals are cases for f.2257

In [97]:
hearing_noise_controls <- hearing_noise %>% 
  filter(cases == FALSE)
head(hearing_noise_controls)
cat(dim(hearing_noise_controls)[1], "individuals are controls for f.2257")

,IID,FID,f.22001.0.0,f.21003.0.0,f.21003.1.0,f.21003.2.0,f.21003.3.0,f.2257.0.0_recode,f.2257.1.0_recode,f.2257.2.0_recode,f.2257.3.0_recode,cases
,<int>,<int>,<fct>,<int>,<int>,<int>,<int>,<fct>,<fct>,<fct>,<fct>,<lgl>
1,1000063,1000063,Male,43,NA,NA,NA,0,NA,NA,NA,FALSE
2,1000078,1000078,Female,52,57,60,NA,0,0,0,NA,FALSE
3,1000129,1000129,Male,62,NA,NA,NA,0,NA,NA,NA,FALSE
4,1000137,1000137,Female,46,NA,NA,NA,0,NA,NA,NA,FALSE
5,1000141,1000141,Female,49,NA,NA,NA,0,NA,NA,NA,FALSE
6,1000186,1000186,Male,53,NA,NA,NA,0,NA,NA,NA,FALSE


210733 individuals are controls for f.2257

In [98]:
dim(hearing_noise)[1]-210733 - 132872

[1] 0

## f.2257 Recode cases=1 and controls=0

In [99]:
hearing_noise$hearing_noise_cat <- as.integer(as.logical(hearing_noise$cases))
head(hearing_noise, 10)
# Get the number of NAs
length(which(is.na(hearing_noise$hearing_noise_cat)))

,IID,FID,f.22001.0.0,f.21003.0.0,f.21003.1.0,f.21003.2.0,f.21003.3.0,f.2257.0.0_recode,f.2257.1.0_recode,f.2257.2.0_recode,f.2257.3.0_recode,cases,hearing_noise_cat
,<int>,<int>,<fct>,<int>,<int>,<int>,<int>,<fct>,<fct>,<fct>,<fct>,<lgl>,<int>
1,1000019,1000019,Female,47,NA,NA,NA,1,NA,NA,NA,TRUE,1
2,1000022,1000022,Male,53,NA,NA,NA,1,NA,NA,NA,TRUE,1
3,1000035,1000035,Male,63,NA,NA,NA,1,NA,NA,NA,TRUE,1
4,1000046,1000046,Female,62,NA,73,NA,0,NA,1,NA,TRUE,1
5,1000054,1000054,Female,65,NA,NA,NA,1,NA,NA,NA,TRUE,1
6,1000063,1000063,Male,43,NA,NA,NA,0,NA,NA,NA,FALSE,0
7,1000078,1000078,Female,52,57,60,NA,0,0,0,NA,FALSE,0
8,1000105,1000105,Female,54,NA,NA,NA,1,NA,NA,NA,TRUE,1
9,1000112,1000112,Male,58,NA,68,NA,0,NA,1,NA,TRUE,1


[1] 0

## f.2257 Step 2: get the ages for hearing difficulty with background noise

In [100]:
# Get the subset data of cases
noise_age_cases = hearing_noise %>% 
  filter(hearing_noise_cat=="1")  %>%
  select(IID, f.2257.0.0_recode, f.2257.1.0_recode, f.2257.2.0_recode, f.2257.3.0_recode, f.21003.0.0, f.21003.1.0, f.21003.2.0, f.21003.3.0)
head(noise_age_cases,12)

,IID,f.2257.0.0_recode,f.2257.1.0_recode,f.2257.2.0_recode,f.2257.3.0_recode,f.21003.0.0,f.21003.1.0,f.21003.2.0,f.21003.3.0
,<int>,<fct>,<fct>,<fct>,<fct>,<int>,<int>,<int>,<int>
1,1000019,1,NA,NA,NA,47,NA,NA,NA
2,1000022,1,NA,NA,NA,53,NA,NA,NA
3,1000035,1,NA,NA,NA,63,NA,NA,NA
4,1000046,0,NA,1,NA,62,NA,73,NA
5,1000054,1,NA,NA,NA,65,NA,NA,NA
6,1000105,1,NA,NA,NA,54,NA,NA,NA
7,1000112,0,NA,1,NA,58,NA,68,NA
8,1000164,1,NA,NA,NA,51,NA,NA,NA
9,1000170,1,NA,NA,NA,51,NA,NA,NA


In [101]:
# Get the # of column where first replied Yes:
noise_age_cases$visit_idx = apply(noise_age_cases, 1, function(x) unlist(which(x == '1')))
head(noise_age_cases)

,IID,f.2257.0.0_recode,f.2257.1.0_recode,f.2257.2.0_recode,f.2257.3.0_recode,f.21003.0.0,f.21003.1.0,f.21003.2.0,f.21003.3.0,visit_idx
,<int>,<fct>,<fct>,<fct>,<fct>,<int>,<int>,<int>,<int>,<list>
1,1000019,1,NA,NA,NA,47,NA,NA,NA,2
2,1000022,1,NA,NA,NA,53,NA,NA,NA,2
3,1000035,1,NA,NA,NA,63,NA,NA,NA,2
4,1000046,0,NA,1,NA,62,NA,73,NA,4
5,1000054,1,NA,NA,NA,65,NA,NA,NA,2
6,1000105,1,NA,NA,NA,54,NA,NA,NA,2


In [102]:
# Define offset:
# offset: refers to the # of columns between the first age column (i.e.f.21003.0.0) and the first recode column (i.e.f.2257.0.0_recode)
offset = which(colnames(noise_age_cases) == 'f.21003.0.0') - which(colnames(noise_age_cases) == 'f.2257.0.0_recode')

# Define the function to extract the first time they said yes for cases 
f=get_age_func <- function(x) {
  visit_index=x[which(colnames(noise_age_cases)=="visit_idx")]
  index=min(unlist(visit_index))+offset
  age=x[index]
  final_age=unlist(age)
  if(is.null(final_age))
  {final_age<-NA}
  return(final_age)
}

# Get the final age for cases
noise_age_cases$age_final_noise = apply(noise_age_cases, 1, f)

# Show first 6 rows
head(noise_age_cases)
summary(noise_age_cases$age_final_noise)

,IID,f.2257.0.0_recode,f.2257.1.0_recode,f.2257.2.0_recode,f.2257.3.0_recode,f.21003.0.0,f.21003.1.0,f.21003.2.0,f.21003.3.0,visit_idx,age_final_noise
,<int>,<fct>,<fct>,<fct>,<fct>,<int>,<int>,<int>,<int>,<list>,<int>
1,1000019,1,NA,NA,NA,47,NA,NA,NA,2,47
2,1000022,1,NA,NA,NA,53,NA,NA,NA,2,53
3,1000035,1,NA,NA,NA,63,NA,NA,NA,2,63
4,1000046,0,NA,1,NA,62,NA,73,NA,4,73
5,1000054,1,NA,NA,NA,65,NA,NA,NA,2,65
6,1000105,1,NA,NA,NA,54,NA,NA,NA,2,54


   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  40.00   53.00   60.00   58.56   65.00   82.00 

In [103]:
### Extract age for Control (2257)
noise_age_control <- hearing_noise %>% 
  filter(hearing_noise_cat == "0") %>%
  select(IID, f.2257.0.0_recode, f.2257.1.0_recode, f.2257.2.0_recode, f.2257.3.0_recode, f.21003.0.0, f.21003.1.0, f.21003.2.0, f.21003.3.0)
head(noise_age_control)

,IID,f.2257.0.0_recode,f.2257.1.0_recode,f.2257.2.0_recode,f.2257.3.0_recode,f.21003.0.0,f.21003.1.0,f.21003.2.0,f.21003.3.0
,<int>,<fct>,<fct>,<fct>,<fct>,<int>,<int>,<int>,<int>
1,1000063,0,NA,NA,NA,43,NA,NA,NA
2,1000078,0,0,0,NA,52,57,60,NA
3,1000129,0,NA,NA,NA,62,NA,NA,NA
4,1000137,0,NA,NA,NA,46,NA,NA,NA
5,1000141,0,NA,NA,NA,49,NA,NA,NA
6,1000186,0,NA,NA,NA,53,NA,NA,NA


In [104]:
# Get the # of column where last replied No:
noise_age_control$visit_idx = apply(noise_age_control, 1, function(x) unlist(which(x == '0')))

# Define offset:
# offset: refers to the # of columns between the first age column (i.e.f.21003.0.0) and the first recode column (i.e.f.2257.0.0_recode)
offset = which(colnames(noise_age_control) == 'f.21003.0.0') - which(colnames(noise_age_control) == 'f.2257.0.0_recode')

# Define the function to extract the last time they said no for control

f=get_age_func <- function(x) {
  visit_index=x[which(colnames(noise_age_control)=="visit_idx")]
  index=max(unlist(visit_index))+offset
  age=x[index]
  age=unlist(age)
  return(age)
}

# Get the final age for controls
noise_age_control$age_final_noise = apply(noise_age_control, 1, f)

# Show first 6 rows
head(noise_age_control)
summary(noise_age_control$age_final_noise)

,IID,f.2257.0.0_recode,f.2257.1.0_recode,f.2257.2.0_recode,f.2257.3.0_recode,f.21003.0.0,f.21003.1.0,f.21003.2.0,f.21003.3.0,visit_idx,age_final_noise
,<int>,<fct>,<fct>,<fct>,<fct>,<int>,<int>,<int>,<int>,<list>,<int>
1,1000063,0,NA,NA,NA,43,NA,NA,NA,2,43
2,1000078,0,0,0,NA,52,57,60,NA,"2, 3, 4",60
3,1000129,0,NA,NA,NA,62,NA,NA,NA,2,62
4,1000137,0,NA,NA,NA,46,NA,NA,NA,2,46
5,1000141,0,NA,NA,NA,49,NA,NA,NA,2,49
6,1000186,0,NA,NA,NA,53,NA,NA,NA,2,53


   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
   39.0    50.0    58.0    56.9    64.0    82.0 

In [105]:
# Merge age_cases and age_controls
hearing_noise_age <- rbind(noise_age_cases, noise_age_control) 
dim(hearing_noise_age)
head(hearing_noise_age)

[1] 343605     11

,IID,f.2257.0.0_recode,f.2257.1.0_recode,f.2257.2.0_recode,f.2257.3.0_recode,f.21003.0.0,f.21003.1.0,f.21003.2.0,f.21003.3.0,visit_idx,age_final_noise
,<int>,<fct>,<fct>,<fct>,<fct>,<int>,<int>,<int>,<int>,<list>,<int>
1,1000019,1,NA,NA,NA,47,NA,NA,NA,2,47
2,1000022,1,NA,NA,NA,53,NA,NA,NA,2,53
3,1000035,1,NA,NA,NA,63,NA,NA,NA,2,63
4,1000046,0,NA,1,NA,62,NA,73,NA,4,73
5,1000054,1,NA,NA,NA,65,NA,NA,NA,2,65
6,1000105,1,NA,NA,NA,54,NA,NA,NA,2,54


In [106]:
hearing_noise_age = merge(x = hearing_noise ,y = hearing_noise_age [,c("IID","age_final_noise")],by="IID", all.x=TRUE)
head(hearing_noise_age)
dim(hearing_noise_age)

,IID,FID,f.22001.0.0,f.21003.0.0,f.21003.1.0,f.21003.2.0,f.21003.3.0,f.2257.0.0_recode,f.2257.1.0_recode,f.2257.2.0_recode,f.2257.3.0_recode,cases,hearing_noise_cat,age_final_noise
,<int>,<int>,<fct>,<int>,<int>,<int>,<int>,<fct>,<fct>,<fct>,<fct>,<lgl>,<int>,<int>
1,1000019,1000019,Female,47,NA,NA,NA,1,NA,NA,NA,TRUE,1,47
2,1000022,1000022,Male,53,NA,NA,NA,1,NA,NA,NA,TRUE,1,53
3,1000035,1000035,Male,63,NA,NA,NA,1,NA,NA,NA,TRUE,1,63
4,1000046,1000046,Female,62,NA,73,NA,0,NA,1,NA,TRUE,1,73
5,1000054,1000054,Female,65,NA,NA,NA,1,NA,NA,NA,TRUE,1,65
6,1000063,1000063,Male,43,NA,NA,NA,0,NA,NA,NA,FALSE,0,43


[1] 343605     14

## Merge f.3393, f.2247 and f.2257

In [107]:
#Merge with complete database keep the all the rows from original db
df_3393_2247_2257 = merge(x = df_3393_2247, y = hearing_noise_age [,c("IID","hearing_noise_cat","age_final_noise")],by="IID", all.x=TRUE)
dim(df_3393_2247_2257)
head(df_3393_2247_2257)

[1] 354347   1096

,IID,FID,ignore1,ignore2,ignore3,ignore4,f.31.0.0,f.34.0.0,f.53.0.0,f.53.1.0,⋯,f.3393.3.0_recode,hearing_aid_cat,age_final_aid,hearing_diff_cat,age_final_diff,hearing_diff_cat_new,hearing_aid_cat_new,hearing_diff_new,hearing_noise_cat,age_final_noise
,<int>,<int>,<int>,<int>,<int>,<int>,<fct>,<int>,<fct>,<fct>,⋯,<fct>,<int>,<int>,<int>,<int>,<fct>,<fct>,<fct>,<int>,<int>
1,1000019,1000019,0,0,2,-9,Female,1960,2008-01-24,NA,⋯,NA,0,47,1,47,case,control,1,1,47
2,1000022,1000022,0,0,1,-9,Male,1954,2008-01-22,NA,⋯,NA,0,53,1,53,case,control,1,1,53
3,1000035,1000035,0,0,1,-9,Male,1944,2007-11-08,NA,⋯,NA,0,63,0,63,control,control,0,1,63
4,1000046,1000046,0,0,2,-9,Female,1946,2008-12-01,NA,⋯,NA,0,73,0,73,control,control,0,1,73
5,1000054,1000054,0,0,2,-9,Female,1942,2007-11-23,NA,⋯,NA,0,65,0,65,control,control,0,1,65
6,1000063,1000063,0,0,1,-9,Male,1967,2010-06-26,NA,⋯,NA,0,43,0,43,control,control,0,0,43


## Export file for hearing with background noise f.2257

In [108]:
f.2257_pheno <- df_3393_2247_2257 %>%
    filter(!(is.na(hearing_noise_cat))) %>%
    select(FID, IID, age_final_noise, sex, hearing_noise_cat)
head(f.2257_pheno)
dim(f.2257_pheno)

,FID,IID,age_final_noise,sex,hearing_noise_cat
,<int>,<int>,<int>,<fct>,<int>
1,1000019,1000019,47,1,1
2,1000022,1000022,53,0,1
3,1000035,1000035,63,0,1
4,1000046,1000046,73,1,1
5,1000054,1000054,65,1,1
6,1000063,1000063,43,0,0


[1] 343605      5

In [109]:
write.table(f.2257_pheno, '200828_UKBB_Hearing_background_noise_f2257', quote = FALSE, row.names = FALSE)

## Combined phenotype f.2247 and f.2257

In [114]:
hybrid_inconsistent <- df_3393_2247_2257 %>%
 filter(hearing_noise_cat != hearing_diff_new)
head(hybrid_inconsistent)
cat ("There are ",nrow(hybrid_inconsistent),"individuals inconsistent for f.2247 and f.2257")

,IID,FID,ignore1,ignore2,ignore3,ignore4,f.31.0.0,f.34.0.0,f.53.0.0,f.53.1.0,⋯,f.3393.3.0_recode,hearing_aid_cat,age_final_aid,hearing_diff_cat,age_final_diff,hearing_diff_cat_new,hearing_aid_cat_new,hearing_diff_new,hearing_noise_cat,age_final_noise
,<int>,<int>,<int>,<int>,<int>,<int>,<fct>,<int>,<fct>,<fct>,⋯,<fct>,<int>,<int>,<int>,<int>,<fct>,<fct>,<fct>,<int>,<int>
1,1000035,1000035,0,0,1,-9,Male,1944,2007-11-08,NA,⋯,NA,0,63,0,63,control,control,0,1,63
2,1000046,1000046,0,0,2,-9,Female,1946,2008-12-01,NA,⋯,NA,0,73,0,73,control,control,0,1,73
3,1000054,1000054,0,0,2,-9,Female,1942,2007-11-23,NA,⋯,NA,0,65,0,65,control,control,0,1,65
4,1000164,1000164,0,0,2,-9,Female,1958,2009-06-26,NA,⋯,NA,0,51,0,51,control,control,0,1,51
5,1000318,1000318,0,0,1,-9,Male,1955,2008-02-28,NA,⋯,NA,0,52,0,52,control,control,0,1,52
6,1000745,1000745,0,0,1,-9,Male,1949,2009-11-19,NA,⋯,NA,0,60,1,60,case,control,1,0,60


There are  60855 individuals inconsistent for f.2247 and f.2257

In [115]:
length(which(!is.na(df_3393_2247_2257$hearing_noise_cat) & !is.na(df_3393_2247_2257$hearing_diff_new)))

[1] 329501

In [116]:
hybrid_consistent <- df_3393_2247_2257 %>%
 filter(hearing_noise_cat == hearing_diff_new)
head(hybrid_consistent)
cat ("There are ",nrow(hybrid_consistent),"individuals consistent for f.2247 and f.2257")

,IID,FID,ignore1,ignore2,ignore3,ignore4,f.31.0.0,f.34.0.0,f.53.0.0,f.53.1.0,⋯,f.3393.3.0_recode,hearing_aid_cat,age_final_aid,hearing_diff_cat,age_final_diff,hearing_diff_cat_new,hearing_aid_cat_new,hearing_diff_new,hearing_noise_cat,age_final_noise
,<int>,<int>,<int>,<int>,<int>,<int>,<fct>,<int>,<fct>,<fct>,⋯,<fct>,<int>,<int>,<int>,<int>,<fct>,<fct>,<fct>,<int>,<int>
1,1000019,1000019,0,0,2,-9,Female,1960,2008-01-24,NA,⋯,NA,0,47,1,47,case,control,1,1,47
2,1000022,1000022,0,0,1,-9,Male,1954,2008-01-22,NA,⋯,NA,0,53,1,53,case,control,1,1,53
3,1000063,1000063,0,0,1,-9,Male,1967,2010-06-26,NA,⋯,NA,0,43,0,43,control,control,0,0,43
4,1000078,1000078,0,0,2,-9,Female,1955,2007-08-20,2012-11-01,⋯,NA,0,60,0,60,control,control,0,0,60
5,1000112,1000112,0,0,1,-9,Male,1949,2008-04-01,NA,⋯,NA,1,68,1,68,case,case,1,1,68
6,1000129,1000129,0,0,1,-9,Male,1946,2009-03-20,NA,⋯,NA,NA,NA,0,62,control,NA,0,0,62


There are  268646 individuals consistent for f.2247 and f.2257

In [117]:
age_consistency <- hybrid_consistent %>%
    select(IID, age_final_diff, age_final_noise)
head(age_consistency)

,IID,age_final_diff,age_final_noise
,<int>,<int>,<int>
1,1000019,47,47
2,1000022,53,53
3,1000063,43,43
4,1000078,60,60
5,1000112,68,68
6,1000129,62,62


In [118]:
# There are inconsistencies for the age between hearing difficulty and background noise
all(age_consistency$age_final_diff == age_consistency$age_final_noise)

[1] FALSE

In [119]:
age_inconsistency <- hybrid_consistent %>%
    filter(age_final_diff != age_final_noise)
head(age_inconsistency)
cat ("There are",nrow(age_inconsistency), "individuals with inconsistent ages for hearing difficulty and background noise")

,IID,FID,ignore1,ignore2,ignore3,ignore4,f.31.0.0,f.34.0.0,f.53.0.0,f.53.1.0,⋯,f.3393.3.0_recode,hearing_aid_cat,age_final_aid,hearing_diff_cat,age_final_diff,hearing_diff_cat_new,hearing_aid_cat_new,hearing_diff_new,hearing_noise_cat,age_final_noise
,<int>,<int>,<int>,<int>,<int>,<int>,<fct>,<int>,<fct>,<fct>,⋯,<fct>,<int>,<int>,<int>,<int>,<fct>,<fct>,<fct>,<int>,<int>
1,1002378,1002378,0,0,1,-9,Male,1950,2008-11-01,NA,⋯,NA,0,68,1,58,case,control,1,1,68
2,1006407,1006407,0,0,2,-9,Female,1949,2008-09-30,NA,⋯,NA,0,65,1,59,case,control,1,1,65
3,1006513,1006513,0,0,2,-9,Female,1955,2010-02-23,NA,⋯,NA,1,59,1,59,case,case,1,1,54
4,1008266,1008266,0,0,1,-9,Male,1949,2008-04-30,NA,⋯,NA,0,69,1,58,case,control,1,1,69
5,1012730,1012730,0,0,1,-9,Male,1947,2010-05-26,NA,⋯,0,0,73,1,70,case,control,1,1,63
6,1015865,1015865,0,0,2,-9,Female,1948,2010-04-27,NA,⋯,NA,0,71,1,71,case,control,1,1,62


There are 3278 individuals with inconsistent ages for hearing difficulty and background noise

In [130]:
age_inconsistency_cases <- hybrid_consistent %>%
    filter(hearing_noise_cat == 1 &  hearing_diff_new ==1) %>%
    filter(age_final_diff != age_final_noise) %>%
    select(FID, IID, sex, hearing_diff_new, hearing_noise_cat, age_final_diff, age_final_noise)
head(age_inconsistency_cases)
cat ("There are",nrow(age_inconsistency_cases), "cases with inconsistent ages for hearing difficulty and background noise")

,FID,IID,sex,hearing_diff_new,hearing_noise_cat,age_final_diff,age_final_noise
,<int>,<int>,<fct>,<fct>,<int>,<int>,<int>
1,1002378,1002378,0,1,1,58,68
2,1006407,1006407,1,1,1,59,65
3,1006513,1006513,1,1,1,59,54
4,1008266,1008266,0,1,1,58,69
5,1012730,1012730,0,1,1,70,63
6,1015865,1015865,1,1,1,71,62


There are 3271 cases with inconsistent ages for hearing difficulty and background noise

In [131]:
age_inconsistency_controls <- hybrid_consistent %>%
    filter(hearing_noise_cat == 0 &  hearing_diff_new == 0) %>%
    filter(age_final_diff != age_final_noise) %>%
    select(FID, IID, sex, hearing_diff_new, hearing_noise_cat, age_final_diff, age_final_noise)
head(age_inconsistency_controls)
cat ("There are",nrow(age_inconsistency_controls), "controls with inconsistent ages for hearing difficulty and background noise")

,FID,IID,sex,hearing_diff_new,hearing_noise_cat,age_final_diff,age_final_noise
,<int>,<int>,<fct>,<fct>,<int>,<int>,<int>
1,1469184,1469184,0,0,0,70,63
2,1740024,1740024,0,0,0,41,46
3,2876630,2876630,1,0,0,65,75
4,3618107,3618107,0,0,0,59,69
5,3854162,3854162,0,0,0,50,48
6,4366635,4366635,0,0,0,57,46


There are 7 controls with inconsistent ages for hearing difficulty and background noise

In [137]:
#get the minimum age for 2247 and 2257 for cases
age_inconsistency_cases <- transform(age_inconsistency_cases, age_diff_noise = pmin(age_final_noise, age_final_diff))
head(age_inconsistency_cases)

,FID,IID,sex,hearing_diff_new,hearing_noise_cat,age_final_diff,age_final_noise,age_diff_noise
,<int>,<int>,<fct>,<fct>,<int>,<int>,<int>,<int>
1,1002378,1002378,0,1,1,58,68,58
2,1006407,1006407,1,1,1,59,65,59
3,1006513,1006513,1,1,1,59,54,54
4,1008266,1008266,0,1,1,58,69,58
5,1012730,1012730,0,1,1,70,63,63
6,1015865,1015865,1,1,1,71,62,62


In [138]:
#get the maximun age for 2247 and 2257 for controls
age_inconsistency_controls <- transform(age_inconsistency_controls, age_diff_noise = pmax(age_final_noise, age_final_diff))
head(age_inconsistency_controls,10)

,FID,IID,sex,hearing_diff_new,hearing_noise_cat,age_final_diff,age_final_noise,age_diff_noise
,<int>,<int>,<fct>,<fct>,<int>,<int>,<int>,<int>
1,1469184,1469184,0,0,0,70,63,70
2,1740024,1740024,0,0,0,41,46,46
3,2876630,2876630,1,0,0,65,75,75
4,3618107,3618107,0,0,0,59,69,69
5,3854162,3854162,0,0,0,50,48,50
6,4366635,4366635,0,0,0,57,46,57
7,5668953,5668953,1,0,0,74,69,74


In [144]:
age_solved <- rbind(age_inconsistency_cases, age_inconsistency_controls)
nrow(age_solved)
head(age_solved)

[1] 3278

,FID,IID,sex,hearing_diff_new,hearing_noise_cat,age_final_diff,age_final_noise,age_diff_noise
,<int>,<int>,<fct>,<fct>,<int>,<int>,<int>,<int>
1,1002378,1002378,0,1,1,58,68,58
2,1006407,1006407,1,1,1,59,65,59
3,1006513,1006513,1,1,1,59,54,54
4,1008266,1008266,0,1,1,58,69,58
5,1012730,1012730,0,1,1,70,63,63
6,1015865,1015865,1,1,1,71,62,62


In [143]:
hybrid_age_consistent <- hybrid_consistent  %>%
    filter(age_final_diff == age_final_noise) %>%
    select(FID, IID, sex, hearing_diff_new, hearing_noise_cat, age_final_diff, age_final_noise) %>%
    mutate(age_diff_noise = age_final_diff) 
head(hybrid_age_consistent)
cat ("There are ",nrow(hybrid_age_consistent),"individuals consistent for age and phenotype for f.2247 and f.2257")

,FID,IID,sex,hearing_diff_new,hearing_noise_cat,age_final_diff,age_final_noise,age_diff_noise
,<int>,<int>,<fct>,<fct>,<int>,<int>,<int>,<int>
1,1000019,1000019,1,1,1,47,47,47
2,1000022,1000022,0,1,1,53,53,53
3,1000063,1000063,0,0,0,43,43,43
4,1000078,1000078,1,0,0,60,60,60
5,1000112,1000112,0,1,1,68,68,68
6,1000129,1000129,0,0,0,62,62,62


There are  265368 individuals consistent for age and phenotype for f.2247 and f.2257

In [150]:
f2247_f2257_pheno <- rbind (hybrid_age_consistent, age_solved)
head(f2247_f2257_pheno)
nrow(f2247_f2257_pheno)

,FID,IID,sex,hearing_diff_new,hearing_noise_cat,age_final_diff,age_final_noise,age_diff_noise
,<int>,<int>,<fct>,<fct>,<int>,<int>,<int>,<int>
1,1000019,1000019,1,1,1,47,47,47
2,1000022,1000022,0,1,1,53,53,53
3,1000063,1000063,0,0,0,43,43,43
4,1000078,1000078,1,0,0,60,60,60
5,1000112,1000112,0,1,1,68,68,68
6,1000129,1000129,0,0,0,62,62,62


[1] 268646

In [151]:
names(f2247_f2257_pheno)[names(f2247_f2257_pheno) == 'hearing_diff_new'] <- 'f2247_f2257'
names(f2247_f2257_pheno)[names(f2247_f2257_pheno) == 'age_diff_noise'] <- 'age'
f2247_f2257_pheno <- f2247_f2257_pheno %>%
 select(FID, IID, age, sex, f2247_f2257 )
head(f2247_f2257_pheno)

,FID,IID,age,sex,f2247_f2257
,<int>,<int>,<int>,<fct>,<fct>
1,1000019,1000019,47,1,1
2,1000022,1000022,53,0,1
3,1000063,1000063,43,0,0
4,1000078,1000078,60,1,0
5,1000112,1000112,68,0,1
6,1000129,1000129,62,0,0


In [152]:
write.table(f2247_f2257_pheno, '200828_UKBB_f2247_f2257', quote = FALSE, row.names = FALSE)